In [12]:
import time
import pymongo
import pprint
from pymongo.errors import WriteError, DuplicateKeyError
from pymongo import MongoClient
from datetime import datetime
import os
client = MongoClient('mongodb://localhost:27017')
db = client.abaco
store2 = db['5']

# Destroy Mongo

In [4]:
import multiprocessing
from multiprocessing import Queue

In [84]:
def threaded_thing(nonce_store):
    print('hey')
    res = nonce_store.update_one(
        {'_id': nonce_key, nonce_id + '.remaining_uses': {'$eq': -1}},
        {'$inc': {nonce_id + '.current_uses': 1},
        '$set': {nonce_id + '.last_use_time': 'hey'}})
    if res.raw_result['updatedExisting'] == True:
        logger.debug("nonce has infinite uses. updating nonce.")
        return 'hey!'

    # Check for remaining uses greater than 0
    res = nonce_store.update_one(
        {'_id': nonce_key, nonce_id + '.remaining_uses': {'$gt': 0}},
        {'$inc': {nonce_id + '.current_uses': 1,
                nonce_id + '.remaining_uses': -1},
        '$set': {nonce_id + '.last_use_time': 'hey'}})

In [6]:
nonce_key = 'DEV-DEVELOP_jane'
nonce_id = 'DEV-DEVELOP_BPbPk0xvz0Llj'
print('hey')
res = nonce_store.update_one(
    {'_id': nonce_key, nonce_id + '.remaining_uses': {'$eq': -1}},
    {'$inc': {nonce_id + '.current_uses': 1},
    '$set': {nonce_id + '.last_use_time': 'hey'}})

hey


In [14]:
nonce_store.update_one(
    {'_id':'did'},
    {'$set':{'stopper':'dopper'}},
    upsert=True)

In [24]:
list(actors_store.find({}))

[{'_id': 'bop', 'hey': 'there'}, {'_id': 'dop', 'hey': 'there'}]

In [23]:
actors_store.update_one(
    {'_id': 'dop'},
    {'$set':{'hey':'there'}},
    upsert=True)

In [11]:
def newBoy(num):
    print(num)
    return(num)

In [ ]:
pool = multiprocessing.Pool(processes=4)
hey = pool.map(newBoy, range(128))
pool.join()
pool.close()

In [176]:
nonce_store.update_one(
    {'_id': 'DEV-DEVELOP_jane'},
    {'$set': {'DEV-DEVELOP_KvVArG21Y1GJ': {'cat':2}}})

## Mongo Store

In [571]:
#### Things to implement:
## getset needs to be fixed for non-maze
## add_if_empty. These two just need to be checked. They work, but errors when you're creating new documents with not yet made fields.

### Store class only:
## set_with_expiry
## insert_one?
## set_with_expiry


### maze
## __setitem__
## __getitem__
## __delitem__
## __len__
## _prepset
## pop_field
## getset
## add_if_empty
## set_with_expiry




###### Need to finish
## insert_one?
## __iter__


class testStore():
    def __init__(self):
        db = client.abaco
        test = db['5']
        self._db = test
        
    def __getitem__(self, key):
        return mongoMaze(self._db, key, [])
        
    def __setitem__(self, key, value):
        self._db.update_one(
            filter={'_id': key},
            update={'$set': {key: value}})
        
    def __delitem__(self, key):
        self._db.delete_one({'_id': key})
    
    def __len__(self):
        return self._db.estimated_document_count()
    
    def __repr__(self):
        return pprint.pformat(list(self._db.find(projection={'_id': False})))
    
    def set_with_expiry(self, key, field, value):
        self._db.update_one(
            filter={'_id': key},
            update={'$set': {'exp': datetime.utcnow(), field: self._prepset(value)}},
            upsert=True)
    
    def _prepset(self, value):
        if type(value) is bytes:
            return value.decode('utf-8')
        return value


# MongoCrawl?
class mongoMaze():
    def __init__(self, db, key, field):
        self._db = db
        self.key = key
        self.field = field
        
        _id = key
        fields -> doc1.subdoc2.subsubdoc3
        
    def __getitem__(self, field):
        self.field.append(field)
        return mongoMaze(self._db, self.key, self.field)
    
    def __setitem__(self, field, value):
        search = self._get_search(field)
        self._db.update_one(
            filter={'_id': self.key},
            update={'$set': {search: value}},
            upsert=True)
        
    def __delitem__(self, field):
        search = self._get_search(field)
        self._db.update_one(
            filter={'_id': self.key},
            update={'$unset': {f'{search}': ''}})
        
    def __len__(self):
        result = self._db.find_one(
            {'_id': self.key},
            projection={'_id': False})
        return len(eval('result' + self._get_index()))
        
    def __repr__(self):
        result = self._db.find_one(
            {'_id': self.key},
            projection={'_id': False})
        return pprint.pformat(eval('result' + self._get_index()))
    
    def pop_field(self, field):
        search = self._get_search(field)
        result = self._db.find_one_and_update(
            filter={'_id': self.key},
            update={'$unset': {f'{search}': ''}})
        return pprint.pformat(eval('result' + self._get_index()))
    
    def getset(self, value):
        search = self._get_search()
        result = self._db.find_one_and_update(
            filter={'_id': self.key},
            update={'$set': {search: value}})
        print(self._get_index())
        return eval('result' + self._get_index())
    
    def add_if_empty(self, value):
        search = self._get_search()
        res = self._db.update_one(
            {'_id': self.key},
            {'$setOnInsert': {search: value}},
            upsert=True)
        print(self.key, search, value, res.raw_result)

        if res.upserted_id:
            return field
        else:
            return None
    
    def _get_search(self, field=None):
        if field:
            self.field.append(field)

        if self.field:
            search = '.'.join(self.field)
        else:
            search = self.key
        return search
            
    def _get_index(self):
        if self.field:
            index = "['" + "']['".join(self.field) + "']"
        else:
            index = ''
        return index
    
    def _prepset(self, value):
        if type(value) is bytes:
            return value.decode('utf-8')
        return value
    
testme = testStore()
#This feels wrong, thus it's discontinued. Also I don't know how to fix it.

In [115]:
testme

{'1': {'2': {'newSubDoc': 'a newer hello'}}}

In [110]:
testme = {'1': {'2': {'3': {'4': 'hello.'}}}}

In [111]:
testme

{'1': {'2': {'3': {'4': 'hello.'}}}}

In [112]:
testme['1']['2']['3']['4'] = hello

'hello.'

In [113]:
testme['1']['2'] = {'newSubDoc': 'a newer hello'}

In [114]:
testme

{'1': {'2': {'newSubDoc': 'a newer hello'}}}

# NEW NEW MONGO 2.0 STORE REDUX DELUXE NEW EDITION

In [9]:
#### Things to implement:
## search

#### Need to check:
## __getitem__ (y, y)
## __setitem__ (y, y)
## __delitem__ (y, y)
## __len__ (y, y)
## __repr__ (y)
## __iter__ - No idea the use case, no modifications done
## _process_inputs (y)
## _prepset (y)
## pop (y, y)
## set_with_expiry (y, y)
## add_if_empty (y, y) - With quirks
## getset - (y, y)

class testStore():
    def __init__(self):
        db = client.abaco
        test = db['5']
        self._db = test
        
    def __getitem__(self, fields):
        """
        Atomically does either:
        Gets and returns 'self[key]' or 'self[key][field1][field2][...]' as a dictionary
        """
        key, _, subscripts = self._process_inputs(fields)
        result = self._db.find_one(
            {'_id': key},
            projection={'_id': False})
        if result == None:
            raise KeyError(f"'_id' of '{key}' not found")
        try:
            return eval('result' + subscripts)
        except KeyError:
            raise KeyError(f"Subscript of {subscripts} does not exists in document of '_id' {key}")
        
    def __setitem__(self, fields, value):
        """
        Atomically does either:
        Sets 'self[key] = value' or sets 'self[key][field1][field2][...] = value'
        """
        key, dots, _ = self._process_inputs(fields)
        try:
            if isinstance(fields, str) and isinstance(value, dict):
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$set': value},
                    upsert=True)
            else:
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$set': {dots: value}},
                    upsert=True)
        except WriteError:
            raise WriteError(
                "Likely due to trying to set a subfield of a field that does not exists." +
                "\n Try setting a dict rather than a value. Ex. store['id_key', 'key', 'field'] = {'subfield': 'value'}")
        if result.raw_result['nModified'] == 0:
            if not 'upserted' in result.raw_result:
                raise AttributeError('Field not modified, old value likely the same as new')
        
    def __delitem__(self, fields):
        """
        Atomically does either:
        Deletes 'self[key]'
        Unsets 'self[key][field1][field2][...]'
        """
        key, dots, subscripts = self._process_inputs(fields)
        if not subscripts:
            result = self._db.delete_one({'_id': key})
            if result.raw_result['n'] == 0:
                raise KeyError(f"No document with '_id' of '{key}' found")
        else:
            result = self._db.update_one(
                filter={'_id': key},
                update={'$unset': {f'{dots}': ''}})
            if result.raw_result['nModified'] == 0:
                raise KeyError(f"Subscript of {subscripts} does not exist in document of '_id' {key}")

    def __len__(self):
        """
        Returns the estimated document count of a store to give length
        We don't use '.count_documents()' as it's O(N) versus O(1) of estimated
        Length for a document or subdocument comes from len(store['key']['field1'][...]) using dict len()
        """
        return self._db.estimated_document_count()

    def __repr__(self):
        """
        Returns a pretty string of the entire store with '_id' visible for developer use
        """
        return pprint.pformat(list(self._db.find()))
    
    def __iter__(self):
        for cursor in self._db.find():
            yield cursor['_id']

    
    def _process_inputs(self, fields):
        """
        Takes in fields and returns the key corresponding with '_id', dot notation
        for getting to a specific field in a Mongo query/filter (ex. 'field1.field2.field3.field4')
        and the subscript notation for returning a specified field from a result dictionary
        (ex. `['field1']['field2']['field3']['field4']`)
        """
        if isinstance(fields, str):
            key = dots = fields
            subscripts = ''
        elif isinstance(fields, list) and len(fields) == 1:
            key = dots = fields[0]
            subscripts = ''
        else:
            key = fields[0]
            dots = '.'.join(fields[1:])
            subscripts = "['" + "']['".join(fields[1:]) + "']"
        return key, dots, subscripts
    
    def _prepset(self, value):
        if type(value) is bytes:
            return value.decode('utf-8')
        return value

    def pop_field(self, fields):
        """
        Atomically pops 'self[key] = value' or 'self[key][field1][field2][...] = value'
        """
        key, dots, subscripts = self._process_inputs(fields)
        if not subscripts:
            result = self._db.find_one(
                {'_id': key},
                projection={'_id': False})
            if result == None:
                raise KeyError(f"'_id' of '{key}' not found")
            del_result = self._db.delete_one({'_id': key})
            if del_result.raw_result['n'] == 0:
                raise KeyError(f"No document deleted")
            return result
        else:
            result = self._db.find_one_and_update(
                filter={'_id': key},
                update={'$unset': {dots: ''}})
            try:
                return eval('result' + subscripts)
            except KeyError:
                raise KeyError(f"Subscript of {subscripts} does not exist in document of '_id' {key}")
        
    def set_with_expiry(self, fields, value):
        """
        Atomically:
        Sets 'self[key] = value' or 'self[key][field1][field2][...] = value'
        Creates 'exp' subdocument in document root with current time for use with MongoDB TTL expiration index
        Note: MongoDB TTL checks every 60 secs to delete files
        """
        key, dots, subscripts = self._process_inputs(fields)
        if len(fields) == 1 and isinstance(value, dict):
            result = self._db.update_one(
                filter={'_id': key},
                update={'$set': {'exp': datetime.utcnow()},
                        '$set': value},
                upsert=True)
        else:
            result = self._db.update_one(
                filter={'_id': key},
                update={'$set': {'exp': datetime.utcnow(), dots: self._prepset(value)}},
                upsert=True)
          
    def add_if_empty(self, fields, value):
        """
        Atomically:
        Sets 'self[key] = value' or 'self[key][field1][field2][...] = value'
        Only if the specified key/field(s) combo does not exist or is empty
        Returns the value if it was added; otherwise, returns None
        Note: Will not override a field set to a value in order to create a subfield
        """
        key, dots, _ = self._process_inputs(fields)
        try:
            if len(fields) == 1 and isinstance(value, dict):
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$setOnInsert': value},
                    upsert=True)
                if 'upserted' in result.raw_result:
                    return value
            elif len(fields) == 1:
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$setOnInsert': {dots: value}},
                    upsert=True)
                if 'upserted' in result.raw_result:
                    return value
            else:
                try:
                    result = self._db.update_one(
                        filter={'_id': key, dots: {'$exists': False}},
                        update={'$set': {dots: value}},
                        upsert=True)
                    if result.raw_result['nModified']:
                        return value
                except WriteError:
                    print("Likely due to trying to set a subfield of a field that is already set to one value")
                    pass
            return None
        except DuplicateKeyError:
            return None

    def getset(self, fields, value):
        """
        Atomically does either:
        Sets 'self[key] = value' and returns previous 'self[key]'
        Sets 'self[key][field1][field2][...] = value' and returns previous 'self[key][field1][field2][...]'
        """
        key, dots, subscripts = self._process_inputs(fields)
        result = self._db.find_one_and_update(
            filter={'_id': key, dots: {'$exists': True}},
            update={'$set': {dots: value}})
        if result == None:
            raise KeyError(f"1Subscript of {subscripts} does not exist in document of '_id' {key}")
            
        try:
            if len(fields) == 1:
                return eval(f"result['{key}']")
            else:
                return eval('result' + subscripts)
        except KeyError:
            raise KeyError(f"Subscript of {subscripts} does not exist in document of '_id' {key}")
            
    def search(self, query):
        """
        WIP
        """
        list(test.find({'$text':{'$search': query}}).skip(0).limit(10))
        
        #
        

    def input_checker(start=None, end=None, limit=None, offset=None, **special_flags):
        """
        Checks inputs for multiple functions.
        Checks for negative inputs, correct types,
        start or end independence of limit or offset.
        Also results correct get_db_data inputs for id_input and year input.
        """
        input_errors = []
        if start is not None:
            try:
                start = int(start)
                if start < 0:
                    input_errors.append("Input for 'start' must be positive or 0")
            except ValueError:
                input_errors.append("Input for 'start' must be an int")

        if end is not None:
            try:
                end = int(end)
                if end < 0:
                    input_errors.append("Input for 'end' must be positive or 0")
            except ValueError:
                input_errors.append("Input for 'end' must be an int")

        if start is not None and end is not None:
            if start > end:
                input_errors.append("'start' input cannot be larger than 'end'")

        if limit is not None:
            try:
                limit = int(limit)
                if limit < 0:
                    input_errors.append("Input for 'limit' must be positive or 0")
            except ValueError:
                input_errors.append("Input for 'limit' must be an int")

        if offset is not None:
            try:
                offset = int(offset)
                if offset < 0:
                    input_errors.append("Input for 'offset' must be positive or 0")
            except ValueError:
                input_errors.append("Input for 'offset' must be an int")

        if (start is not None or end is not None) and (limit is not None or offset is not None):
            input_errors.append("Start or end must be used independently of limit or offset")

        if "id_input" in special_flags:
            try:
                id_input = int(special_flags["id_input"])
                if id_input < 0:
                    input_errors.append("Input for 'id' must be positive or 0")
                limit = 1
                offset = id_input
            except ValueError:
                input_errors.append("Input for 'id' must be an int")

        if "year" in special_flags:
            try:
                year = int(special_flags["year"])
                if year < 0:
                    input_errors.append("Input for 'year' must be positive or 0")
                start = year
                end = year
            except ValueError:
                input_errors.append("Input for 'year' must be an int")

        if input_errors:
            return start, end, limit, offset, input_errors

        return start, end, limit, offset, input_errors
        
testme = testStore()

db = client.abaco
test = db['5']

In [ ]:
testme['ACTORID']['DOC2'] = 2
testme.getset(['actorid', 'doc1'], '2')

In [756]:
test.find({'$text':{'$search': 'DEV-DEVELOP'}}).skip(0).limit(10))

[{'4AZgJwX78y31': {'ch_name': 'worker_4AZgJwX78y31',
                   'cid': '14fca2d1d81451c199ccb3e1ea80ee3ac945723d24b3b96cd2f5b0ddf9be861d',
                   'create_time': '1580504137.189891',
                   'host_id': '0',
                   'host_ip': '172.17.0.1',
                   'id': '4AZgJwX78y31',
                   'image': 'jstubbs/abaco_test',
                   'last_execution_time': 0,
                   'last_health_check_time': '1580504137.186432',
                   'location': 'unix://var/run/docker.sock',
                   'status': 'READY',
                   'tenant': 'DEV-DEVELOP'},
  '8V47qxAzDzygq': {'ch_name': 'worker_8V47qxAzDzygq',
                    'cid': 'e8df768df39bc3f3fc71546ad1d9a53122d2998a1411dd36b79a4e34d8b889fb',
                    'create_time': '1580504081.516901',
                    'host_id': '0',
                    'host_ip': '172.17.0.1',
                    'id': '8V47qxAzDzygq',
                    'image': 'jstubbs/abaco

In [730]:
# Do this to create a wildcard index on littttterally everything.
test.create_index('$**')

'$**_1'

In [36]:
for k in range(5):
    print(k)

0
1
2
3
4


In [3]:
dog = {'hey':'there'}

In [4]:
if dog.hey:
    print(dog.hey)

AttributeError: 'dict' object has no attribute 'hey'

In [33]:
result = test.update_one({'_id':'DEV-DEVELOP_ArxxNeJlM6My'},
                {'$set': {'dog': 'hellllddo!'}},
                upsert=True)

In [27]:
testme

[{'_id': 'DEV-DEVELOP_ArxxNeJlM6My',
  'api_server': 'https://dev.tenants.develop.tacc.cloud',
  'create_time': '1580913028.723185',
  'db_id': 'DEV-DEVELOP_ArxxNeJlM6My',
  'default_environment': {},
  'description': '',
  'dog': 'hellllo!',
  'executions': {},
  'gid': None,
  'hints': [],
  'id': 'ArxxNeJlM6My',
  'image': 'abacosamples/test',
  'last_update_time': '1580913028.723185',
  'link': '',
  'max_cpus': None,
  'max_workers': None,
  'mem_limit': None,
  'mounts': [{'container_path': '/_abaco_data1',
              'host_path': '/data1',
              'mode': 'ro'}],
  'name': 'abaco_test_suite_limited_user',
  'owner': 'testotheruser',
  'privileged': False,
  'queue': 'default',
  'state': {},
  'stateless': True,
  'status': 'READY',
  'status_message': ' ',
  'tasdir': None,
  'tenant': 'DEV-DEVELOP',
  'token': 'false',
  'type': 'none',
  'uid': None,
  'use_container_uid': False,
  'webhook': ''},
 {'_id': 'DEV-dsdasdDEVELOP_ArxxNeJlM6My', 'dog': 'hellllo!'}]